In [ ]:
#Framework
import torch
import torch.nn as nn
import torch.nn.functional as F  #torch is the main PyTorch library, nn contains neural network modules, and F contains various functions for neural network operations.

class MLP(nn.Module): #custom class named MLP (Multi-Layer Perceptron) that inherits from nn.Module.  represent neural network model
    def __init__(self, input_size, hidden_size, output_size): # defines the constructor method for the MLP class
        super(MLP, self).__init__() # calls the constructor of the parent class nn.Module to initialize the MLP class
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

def main():
    x = torch.randn(2, 3) #tensor
    model = MLP(3, 4, 2)
    y = model(x)
    print(y)

if __name__ == "__main__":
    main()


tensor([[ 0.3123, -0.7571],
        [ 0.0929, -0.4266]], grad_fn=<AddmmBackward0>)


In [ ]:
#Framework
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define a function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {total_loss / len(train_loader)}")

# Define a function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, targets in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        return accuracy

# Main function
def main():
    # Generate synthetic data for demonstration purposes
    X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize features using StandardScaler (if needed)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert data to PyTorch tensors
    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

    # Create data loaders
    batch_size = 64
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Define model parameters
    input_size = X_train.shape[1]
    hidden_size = 128
    output_size = 2  # Change this to the number of classes in your dataset

    # Create the MLP model
    model = MLP(input_size, hidden_size, output_size)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    num_epochs = 10
    train_model(model, train_loader, criterion, optimizer, num_epochs)

    # Evaluate the model on the test set
    accuracy = evaluate_model(model, test_loader)
    print(f"Test Accuracy: {accuracy:.2f}%")

if __name__ == "__main__":
    main()


Epoch [1/10] Loss: 0.6708698410254258
Epoch [2/10] Loss: 0.5423146302883441
Epoch [3/10] Loss: 0.46120363703140843
Epoch [4/10] Loss: 0.41199647234036374
Epoch [5/10] Loss: 0.37949171433081996
Epoch [6/10] Loss: 0.356646872483767
Epoch [7/10] Loss: 0.34246204793453217
Epoch [8/10] Loss: 0.3305731026025919
Epoch [9/10] Loss: 0.3189007422098747
Epoch [10/10] Loss: 0.3125559091567993
Test Accuracy: 83.50%
